In [8]:
import os
import random
import shutil
import yaml

# Укажите глобальные пути к данным
images_dir = r"C:\vkcv2022-contest-02-carplates\data\images"
labels_dir = r"C:\vkcv2022-contest-02-carplates\data\labels"
output_dir = r"C:\vkcv2022-contest-02-carplates\data\processed"  # Папка для сохранения разделенных данных

# Параметры разделения данных
validation_split = 0.15  # Доля для валидации
test_split = 0.05  # Доля для теста
train_split = 1 - validation_split - test_split  # Доля для обучения

# Проверка существования папок с данными
train_images_dir = os.path.join(output_dir, 'images/train')
val_images_dir = os.path.join(output_dir, 'images/val')
test_images_dir = os.path.join(output_dir, 'images/test')

train_labels_dir = os.path.join(output_dir, 'labels/train')
val_labels_dir = os.path.join(output_dir, 'labels/val')
test_labels_dir = os.path.join(output_dir, 'labels/test')

# Функция для проверки существования и создания папок
def create_dirs():
    os.makedirs(train_images_dir, exist_ok=True)
    os.makedirs(val_images_dir, exist_ok=True)
    os.makedirs(test_images_dir, exist_ok=True)
    os.makedirs(train_labels_dir, exist_ok=True)
    os.makedirs(val_labels_dir, exist_ok=True)
    os.makedirs(test_labels_dir, exist_ok=True)

# Список всех изображений и аннотаций с учетом новых форматов
image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')) and not f.startswith('.')]
label_files = [f.replace(f.split('.')[-1], 'txt') for f in image_files]

# Убедимся, что количество изображений и аннотаций совпадает
assert len(image_files) == len(label_files), "Количество изображений и аннотаций должно совпадать!"

# Отладочная информация: сколько изображений и аннотаций найдено
print(f"Найдено {len(image_files)} изображений и {len(label_files)} аннотаций.")

# Перемешиваем файлы для случайного распределения
data = list(zip(image_files, label_files))
random.shuffle(data)

# Подсчет количества данных в каждой выборке
total_data = len(data)
val_count = int(total_data * validation_split)
test_count = int(total_data * test_split)
train_count = total_data - val_count - test_count  # Остаток идет в обучение

# Отладочная информация: сколько данных в каждой выборке
print(f"Всего данных: {total_data}")
print(f"Обучение: {train_count} данных")
print(f"Валидация: {val_count} данных")
print(f"Тест: {test_count} данных")

# Разделяем данные
train_data = data[:train_count]
val_data = data[train_count:train_count + val_count]
test_data = data[train_count + val_count:]

# Проверка: сумма всех выборок должна быть равна общему количеству данных
assert len(train_data) + len(val_data) + len(test_data) == total_data, "Ошибка: сумма подвыборок не равна общему количеству данных!"

# Отладочная информация: после разделения
print(f"После разделения: {len(train_data)} для обучения, {len(val_data)} для валидации, {len(test_data)} для теста.")

# Функция для копирования файлов в соответствующие директории
def copy_files(data, src_image_dir, src_label_dir, dst_image_dir, dst_label_dir):
    for img_file, lbl_file in data:
        try:
            shutil.copy(os.path.join(src_image_dir, img_file), os.path.join(dst_image_dir, img_file))
            shutil.copy(os.path.join(src_label_dir, lbl_file), os.path.join(dst_label_dir, lbl_file))
        except FileNotFoundError as e:
            print(f"Пропущен файл: {img_file} или {lbl_file} - {e}")
        except Exception as e:
            print(f"Ошибка при копировании файла {img_file} или {lbl_file} - {e}")

# Проверяем, созданы ли папки для разделения, если нет — создаем
if not (os.path.exists(train_images_dir) and os.path.exists(val_images_dir) and os.path.exists(test_images_dir)):
    create_dirs()

    # Копируем данные, если они еще не были скопированы
    copy_files(train_data, images_dir, labels_dir, train_images_dir, train_labels_dir)
    copy_files(val_data, images_dir, labels_dir, val_images_dir, val_labels_dir)
    copy_files(test_data, images_dir, labels_dir, test_images_dir, test_labels_dir)

# Автоматическое извлечение классов из аннотаций YOLO
classes = set()

for lbl_file in label_files:
    label_path = os.path.join(labels_dir, lbl_file)
    try:
        with open(label_path, 'r') as f:
            for line in f.readlines():
                class_id = int(line.split()[0])  # Первое число в строке — это класс
                classes.add(class_id)
    except FileNotFoundError as e:
        print(f"Аннотация не найдена: {lbl_file} - {e}")
    except Exception as e:
        print(f"Ошибка при извлечении данных из файла {lbl_file}: {e}")

# Сортируем классы и создаем список
classes = sorted(list(classes))

# Отладочная информация: сколько классов найдено
print(f"Найдено {len(classes)} классов: {classes}")

# Путь до YAML файла
yaml_path = os.path.join(output_dir, 'data.yaml')

# Данные для файла data.yaml
data_yaml = {
    'train': train_images_dir,
    'val': val_images_dir,
    'test': test_images_dir,
    'nc': len(classes),  # Количество классов
    'names': [f'class{i}' for i in classes]  # Пример: 'class0', 'class1', ...
}

# Сохраняем YAML файл
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(data_yaml, yaml_file, default_flow_style=False)

# Выводим путь к YAML файлу
print(f'Файл data.yaml сохранен по пути: {yaml_path}')


Найдено 25633 изображений и 25633 аннотаций.
Всего данных: 25633
Обучение: 20508 данных
Валидация: 3844 данных
Тест: 1281 данных
После разделения: 20508 для обучения, 3844 для валидации, 1281 для теста.
Пропущен файл: 25632.bmp или 25632.txt - [Errno 2] No such file or directory: 'C:\\vkcv2022-contest-02-carplates\\data\\labels\\25632.txt'
Аннотация не найдена: 25632.txt - [Errno 2] No such file or directory: 'C:\\vkcv2022-contest-02-carplates\\data\\labels\\25632.txt'
Найдено 1 классов: [0]
Файл data.yaml сохранен по пути: C:\vkcv2022-contest-02-carplates\data\processed\data.yaml


In [2]:
from ultralytics import YOLO
import torch


torch.set_num_threads(1)


model = YOLO("yolo11n.pt")
train_results = model.train(
    data=r"C:\Users\user\Desktop\processed\data.yaml",  # path to dataset YAML
    epochs=1000,  # number of training epochs
    imgsz=640,  # training image size
    device=0,  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    patience=100,
    batch=24,
)

New https://pypi.org/project/ultralytics/8.3.49 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.38  Python-3.10.7 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24564MiB)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=C:\Users\user\Desktop\processed\data.yaml, epochs=1000, time=None, patience=100, batch=24, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False

train: Scanning C:\Users\user\Desktop\processed\labels\train.cache... 20507 images, 0 backgrounds, 1 corrupt: 100%|██████████| 20508/20508 [00:00<?, ?it/s]

train: WARNING  C:\Users\user\Desktop\processed\images\train\25632.bmp: ignoring corrupt image/label: cannot identify image file 'C:\\Users\\user\\Desktop\\processed\\images\\train\\25632.bmp'
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\user\Desktop\processed\labels\val.cache... 3844 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3844/3844 [00:00<?, ?it/s]


Plotting labels to runs\detect\train9\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005625000000000001), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train9
Starting training for 1000 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

   292/1000      3.46G     0.6777     0.3302     0.9108         21        640: 100%|██████████| 855/855 [01:41<00:00,  8.42it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 81/81 [00:12<00:00,  6.51it/s]

                   all       3844       4032       0.98      0.979      0.992      0.855
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 192, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



292 epochs completed in 9.481 hours.
Optimizer stripped from runs\detect\train9\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train9\weights\best.pt, 5.5MB

Validating runs\detect\train9\weights\best.pt...
Ultralytics 8.3.38  Python-3.10.7 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24564MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 81/81 [00:14<00:00,  5.48it/s]


                   all       3844       4032      0.979      0.979      0.992      0.856
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\train9
